# Tic-Tac-Toe

In [1]:
from tkinter import *
import math
from tkinter.font import Font
import sys
import copy
import numpy as np
import _pickle as pickle

from keras.models import Sequential
from keras.layers import Dense

/home/crackysk/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Player class

In [2]:
class Player:
    def __init__(self, mark):
        self.mark = mark
        self.opponentMark()

    def opponentMark(self):
        if self.mark == 'X':
            self.opponent = 'O'
        elif self.mark == 'O':
            self.opponent = 'X'
        else:
            sys.exit("Player's mark needs to be 'X' or 'O'.")

    def getMark(self):
        return self.mark

    def mark2num(self):
        if self.mark == 'X':
            return 1
        else:
            return -1

In [3]:
class HumanPlayer(Player):
    pass

In [4]:
class ComputerPlayer(Player):
    def __init__(self, mark, Q={}, epsilon=0.3):
        super(ComputerPlayer, self).__init__(mark=mark)
        self.Q = Q
        self.epsilon = epsilon

    def getMove(self, board):
        if np.random.uniform() < self.epsilon:
            moves = board.availableMoves()
            if moves:   # Make a random move from available ones
                return moves[np.random.choice(len(moves))]
        else:
            stateKey = ComputerPlayer.makeAndAddKey(board, self.mark, self.Q)
            Qs = self.Q[stateKey]

            if self.mark == 'X':
                return ComputerPlayer.stochasticArgminmax(Qs, max)
            else:
                return ComputerPlayer.stochasticArgminmax(Qs, min)

    @staticmethod
    def makeAndAddKey(board, mark, Q):
        defaultQValue = 0.0
        stateKey = board.makeKey(mark)
        if Q.get(stateKey) is None:
            moves = board.availableMoves()
            Q[stateKey] = {move: defaultQValue for move in moves}
        return stateKey

    @staticmethod
    def stochasticArgminmax(Qs, minMax):
        QValue = minMax(Qs.values())
        if sum(value == QValue for value in Qs.values()) > 1:
            # if move options with same reward, we choose random
            bestOptions = [move for move in Qs.keys() if Qs[move] == QValue]
            move = bestOptions[np.random.choice(len(bestOptions))]
        else:
            move = minMax(Qs, key=Qs.get)
        return move

### Board class

In [5]:
boardSizeGlobal = 3
class Board:
    def __init__(self):
        self.emptyText = '--'
        self.empty = np.nan
        self.boardSize = boardSizeGlobal
        self.over = False
        self.winner = None
        self.fields = np.empty((self.boardSize, self.boardSize,))
        self.fields.fill(self.empty)

    def makeMove(self, player, x, y):
        self.fields[x, y] = player.mark2num()
        return self

    def availableMoves(self):
        return [(x, y) for x in range(self.boardSize) for y in range(self.boardSize) if np.isnan(self.fields[x, y])]

    def nextBoard(self, move, player):
        nextBoard = copy.deepcopy(self)
        nextBoard.fields[tuple(move)] = player.mark2num()
        if nextBoard.isWon():
            nextBoard.over = True
            nextBoard.winner = player
        elif nextBoard.isOver():
            nextBoard.over = True
        return nextBoard

    def isOver(self):
        return True if len(self.availableMoves()) == 0 else False

    def isWon(self):
        sumX = 0
        sumO = 0
        winningIndexes = []
        #horizontalWin
        for x in range(self.boardSize):
            for y in range (self.boardSize):
                if self.fields[x, y] == 1:
                    sumX += 1
                    winningIndexes.append((x, y))
                elif self.fields[x, y] == -1:
                    sumO += 1
                    winningIndexes.append((x, y))
            if sumX == self.boardSize or sumO == self.boardSize:
                return winningIndexes
            winningIndexes.clear()
            sumX = 0
            sumO = 0

        #endOfHorizontalWin

        #verticalWin
        sumX = 0
        sumO = 0
        winningIndexes.clear()
        for y in range (self.boardSize):
            for x in range(self.boardSize):
                if self.fields[x, y] == 1:
                    sumX += 1
                    winningIndexes.append((x, y))
                elif self.fields[x, y] == -1:
                    sumO += 1
                    winningIndexes.append((x, y))

            if sumX == self.boardSize or sumO == self.boardSize:
                return winningIndexes
            winningIndexes.clear()
            sumX = 0
            sumO = 0
        # endOfVerticalWin

        #diagonalWin
        sumX = 0
        sumO = 0
        winningIndexes.clear()
        for x in range(self.boardSize):
            y = x
            if self.fields[x, y] == 1:
                sumX += 1
                winningIndexes.append((x, y))
            elif self.fields[x, y] == -1:
                sumO += 1
                winningIndexes.append((x, y))
        if sumX == self.boardSize or sumO == self.boardSize:
            return winningIndexes
        #endOfDiagonalWin

        #otherDiagonalWin
        sumX = 0
        sumO = 0
        winningIndexes.clear()
        y = self.boardSize
        for x in range(self.boardSize):
            y -= 1
            if self.fields[x, y] == 1:
                sumX += 1
                winningIndexes.append((x, y))
            elif self.fields[x, y] == -1:
                sumO += 1
                winningIndexes.append((x, y))
            if sumX == self.boardSize or sumO == self.boardSize:
                return winningIndexes
        #endOfOtherDiagonalWin

    def makeKey(self, mark):
        fillValue = 9
        filledBoard = copy.deepcopy(self.fields)
        np.place(filledBoard, np.isnan(filledBoard), fillValue)
        return "".join(map(str, map(int, filledBoard.flatten()))) + mark

    def giveReward(self):
        if self.over:
            if self.winner:
                if self.winner.getMark() == 'X':
                    return 10
                elif self.winner.getMark() == 'O':
                    return -10
            else:
                return -5
        else:
            return 0

### Game class

In [6]:
class MainGameGui:    
    def __init__(self, player1, player2, QLearn=False, DQNLearn=False, Q={}, alpha=0.8, gamma=0.95, decay_factor=0.999995):
        self.app = Tk()
        self.app.title('Tic-Tac-Toe')
        self.app.resizable(width=True, height=True)
        self.playBoard = Board()
        self.player1 = player1
        self.player2 = player2
        self.current_player = player1
        self.other_player = player2
        self.buttons = {}
        self.winDetected = {}
        self.turnLabel = ''
        self.QLearn = QLearn
        self.DQNLearn = DQNLearn
        if self.QLearn and self.DQNLearn:
            sys.exit('Wrong input')
        self.learn = self.QLearn or self.DQNLearn
        self.Q = Q
        self.alpha = alpha
        self.gamma = gamma
        self.decay_factor = decay_factor
        self.initInterface()
        
        if self.DQNLearn:
            self.model = self.createModel()
        
    def initInterface(self):
        self.chooseSizeLabel = Label(self.app, text='Choose size of a board:',  font=Font(family="Helvetica", size=10, weight='bold'))
        self.turnLabel = Label(self.app, text=self.current_player.getMark() + '\'s turn', font=Font(size=12))
        it = np.nditer(self.playBoard.fields, flags=['multi_index'])
        while not it.finished:
            (x, y) = it.multi_index
            functionClick = lambda x=x, y=y: self.makeMove(x, y)
            buttonClick = Button(self.app, text=self.playBoard.emptyText, font=Font(family="Times", size=15, weight='bold'), command=functionClick, width=10, height=5)
            buttonClick.grid(row=x, column=y)
            self.buttons[x, y] = buttonClick
            it.iternext()
        functionReset = lambda: self.reset()
        buttonReset = Button(self.app, text='reset', command=functionReset, width = 15)
        self.radioButton5 = Radiobutton(self.app, text='5x5', variable = boardSizeGlobal, value = 5, command = functionReset)
        self.radioButton3 = Radiobutton(self.app, text= '3x3', variable = boardSizeGlobal, value = 3, command = functionReset)
        #positioning of the elements in gui
        self.chooseSizeLabel.grid(row=0, column=self.playBoard.boardSize)
        self.radioButton3.grid(row=math.floor(self.playBoard.boardSize/5), column=self.playBoard.boardSize, sticky=S)
        self.radioButton5.grid(row=math.floor(self.playBoard.boardSize/5)+1, column=self.playBoard.boardSize, sticky=N)
        buttonReset.grid(row=self.playBoard.boardSize, column=self.playBoard.boardSize)
        self.turnLabel.grid(row = self.playBoard.boardSize+30, column = 0)
        
    def createModel(self):
        return
        # TODO: konsultacije        
        
    def learnQ(self, move):
        stateKey = ComputerPlayer.makeAndAddKey(self.playBoard, self.current_player.mark, self.Q)
        
        if self.DQNLearn:
            Qs = self.model.predict(stateKey)
            if self.current_player.mark == 'X':
                move = max(Qs, key=Qs.get)
            else:
                move = min(Qs, key=Qs.get)
        
        nextBoard = self.playBoard.nextBoard(move, self.current_player)
        nextReward = nextBoard.giveReward()
        nextStateKey = ComputerPlayer.makeAndAddKey(nextBoard, self.other_player.mark, self.Q)
        
        if nextBoard.isOver():
            expected = nextReward
        else:
            if self.DQNLearn:
                nextQs = self.model.predict(nextStateKey)
            else:
                nextQs = self.Q[nextStateKey]

            if self.current_player.mark == 'X':
                expected = nextReward + (self.gamma * min(nextQs.values()))
            else:
                expected = nextReward + (self.gamma * max(nextQs.values()))
        
            if self.DQNLearn:
                Qs[move] = expected
                self.model.fit(stateKey, Qs, batch_size=1, verbose=0)
            else:
                change = self.alpha * (expected - self.Q[stateKey][move])
                self.Q[stateKey][move] += change
                
    def generateQTable(self):
        for stateKey in self.Q:
            self.Q[stateKey] = model.predict(stateKey)

    def makeMove(self, x, y):
        self.playBoard = self.playBoard.makeMove(self.current_player, x, y)
        (self.current_player, self.other_player) = (self.other_player, self.current_player)

        self.updateStatus()

    def nextMove(self):
        if isinstance(self.current_player, HumanPlayer):
            pass
        else:
            move = self.current_player.getMove(self.playBoard)
            if self.learn:
                self.learnQ(move)
            self.makeMove(move[0], move[1])

    def reset(self):
        if self.DQNLearn:
            self.generateQTable()
        if isinstance(self.player1, ComputerPlayer):
            self.player1.Q = self.Q
            self.player1.epsilon *= self.decay_factor
        if isinstance(self.player2, ComputerPlayer):
            self.player2.Q = self.Q
            self.player2.epsilon *= self.decay_factor
        self.playBoard=Board()
        self.current_player = self.player1
        self.other_player = self.player2
        for x, y in self.buttons:
            self.buttons[x, y]['state'] = 'normal'

        self.updateStatus()

    def updateStatus(self):
        self.turnLabel.configure(text=self.current_player.getMark() + '\'s turn')
        it = np.nditer(self.playBoard.fields, flags=['multi_index'])
        while not it.finished:
            (x, y) = it.multi_index
            if self.playBoard.fields[x, y] == 1:
                text = 'X'
            elif self.playBoard.fields[x, y] == -1:
                text = 'O'
            else:
                text = self.playBoard.emptyText
            self.buttons[x, y]['text'] = text
            self.buttons[x, y]['disabledforeground'] = 'black'
            if text == self.playBoard.emptyText:
                self.buttons[x, y]['state'] = 'normal'
            else:
                self.buttons[x, y]['state'] = 'disabled'
            it.iternext()
        self.winDetected = self.playBoard.isWon()
        if self.winDetected:
            self.playBoard.over = True
            for x, y in self.winDetected:
                self.buttons[x, y]['disabledforeground'] = 'red'

            for x, y in self.buttons:
                self.buttons[x, y]['state'] = 'disabled'
            
            it = np.nditer(self.playBoard.fields, flags=['multi_index'])
            while not it.finished:
                (x, y) = it.multi_index
                it.iternext()
                self.buttons[x, y].update()
            self.playBoard.winner = self.other_player
            self.turnLabel.configure(text=self.other_player.getMark() + ' Won!')
            return
        if self.playBoard.isOver():
            self.playBoard.over = True
            return
        self.nextMove()

    def mainloop(self):
        self.app.mainloop()

### Execution

#### AI vs AI training

In [7]:
num_of_games = 200000
filename = 'Q_table_dictionary_%s.p' % num_of_games

In [8]:
game = MainGameGui(ComputerPlayer('X', epsilon=0.3), ComputerPlayer('O', epsilon=0.3), QLearn=True)
Q={}
for i in range(0, num_of_games):
    game.reset()

# TODO: save model weights
pickle.dump(game.Q, open(filename, 'wb'))

#### AI vs Human

In [10]:
MainGameGui(ComputerPlayer('X', epsilon=0), HumanPlayer('O'), Q=pickle.load(open(filename, 'rb'))).mainloop()

#### Human vs AI

In [11]:
MainGameGui(HumanPlayer('X'), ComputerPlayer('O', epsilon=0), Q=pickle.load(open(filename, 'rb'))).mainloop()

In [9]:
for key in game.Q:
        print(key, game.Q[key], sep='\t')

999999999X	{(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0.0}
999991999O	{(0, 0): 7.737809374999999, (0, 1): 7.737809374999999, (0, 2): 0.0, (1, 0): 0.0, (1, 1): 0.0, (2, 0): 7.737809374999999, (2, 1): 7.737809374999999, (2, 2): 0.0}
999991-199X	{(0, 0): 0.0, (0, 1): -7.735247910767877, (0, 2): -7.73718780700435, (1, 0): -7.737809206933866, (1, 1): 0.0, (2, 1): -7.737809363576059, (2, 2): 8.1450625}
999991-191O	{(0, 0): 9.499999999999213, (0, 1): 9.49999997204562, (0, 2): 8.57375, (1, 0): 9.499999999997334, (1, 1): 9.49999999999997, (2, 1): 9.499999999999764}
999991-1-11X	{(0, 0): 4.6208, (0, 1): 0.0, (0, 2): 10.0, (1, 0): 0.0, (1, 1): 4.6208}
991991-1-11O	{(0, 0): 0.0, (0, 1): 0.0, (1, 0): 0.0, (1, 1): 0.0}
999199999O	{(0, 0): 0.0, (0, 1): 7.737809374999999, (0, 2): 7.737809374999999, (1, 1): 0.0, (1, 2): 0.0, (2, 0): 0.0, (2, 1): 7.737809374999999, (2, 2): 7.737809374999999}
99919999-1X	{(0, 0): -7.737809374850842, (0

99-19-19911X	{(0, 0): -9.499388108788045, (0, 1): -9.499999998393712, (1, 0): -9.48402176, (1, 2): -9.484799999998007, (2, 0): 10.0}
99-11-19911O	{(0, 0): 7.295999999999999, (0, 1): 9.301913599999999, (1, 2): 6.08, (2, 0): -9.9999744}
-19-11-19911X	{(0, 1): 0.0, (1, 2): -9.107839844352, (2, 0): 10.0}
-19-11-19111O	{(0, 1): 0.0, (1, 2): 0.0}
1-1919999-1X	{(0, 2): 0.0, (1, 1): 0.0, (1, 2): 7.106420735999999, (2, 0): 9.999999999997378, (2, 1): 0.0}
1-1919991-1O	{(0, 2): 9.4999745593344, (1, 1): 9.499975518125282, (1, 2): 9.499695513599999, (2, 0): 0.0}
1-191-1991-1X	{(0, 2): 0.0, (1, 2): 0.0, (2, 0): 10.0}
1-191-1911-1O	{(0, 2): 0.0, (1, 2): 0.0}
-111999999O	{(1, 0): -8.1450625, (1, 1): 0.0, (1, 2): 6.532805264408577, (2, 0): -8.064841681631306, (2, 1): 0.0, (2, 2): 8.51022708942591}
-11199199-1O	{(1, 0): -9.024998649615348, (1, 1): -9.999872, (2, 0): 0.0, (2, 1): -8.9712117809152}
19-1919-199X	{(0, 1): 9.025, (1, 0): 8.95279985118804, (1, 2): 7.216137011200001, (2, 1): 7.219934199807999,

1-19-11199-1X	{(0, 2): 0.0, (2, 0): 0.0, (2, 1): 0.0}
-11-19111-19O	{(1, 0): 0.0, (2, 2): 0.0}
1-11-1191-19O	{(1, 2): 0.0, (2, 2): 0.0}
91-19-11-119X	{(0, 0): 0.0, (1, 0): 0.0, (2, 2): 0.0}
1-11-1919-11O	{(1, 1): 0.0, (2, 0): 0.0}
19-1-19-1191X	{(0, 1): -7.5392, (1, 1): 0.0, (2, 1): 9.999998976}
19-11-1-1119O	{(0, 1): 0.0, (2, 2): 0.0}
-1191-11-199X	{(0, 2): -9.5, (2, 1): -9.5, (2, 2): -9.5}
911-1-19991O	{(0, 0): 8.816, (1, 2): -10.0, (2, 0): 9.41183999999761, (2, 1): 7.6000000000000005}
-1-199191-11X	{(0, 2): 9.99999488, (1, 0): -8.815980543999999, (1, 2): -8.512}
1-191-11-191O	{(0, 2): -9.999999999997378, (2, 1): -8.0}
-1-1991911-1X	{(0, 2): 10.0, (1, 0): -9.496959999501843, (1, 2): -9.119513444352}
-1-1191911-1O	{(1, 0): 0.0, (1, 2): 0.0}
11991-19-1-1X	{(0, 2): 10.0, (1, 0): -9.499998837871786, (2, 0): -9.49643001856}
11191-19-1-1O	{(1, 0): 0.0, (2, 0): 0.0}
9-11-11199-1X	{(0, 0): 0.0, (2, 0): 10.0, (2, 1): 0.0}
119-1-19199O	{(0, 2): 0.0, (1, 2): -10.0, (2, 1): 9.498808319999602, (2